In [ ]:
import argparse
from typing import Optional, Union

import pandas as pd
import numpy as np
import torch
import torch.nn as nn

from dataclasses import dataclass

import datasets
from datasets import Dataset

from sklearn.metrics import log_loss

from transformers import (
    AutoTokenizer,
    AutoConfig,
    EarlyStoppingCallback,
    AutoModelForCausalLM,
    AutoModelForMultipleChoice,
    TrainingArguments,
    Trainer,
    RobertaForMultipleChoice,
    AutoModelForSequenceClassification,
    LlamaModel,
    LlamaForSequenceClassification,
    BitsAndBytesConfig,
    get_polynomial_decay_schedule_with_warmup,
    get_cosine_schedule_with_warmup,
    TrainerCallback,
)
from transformers.tokenization_utils_base import PreTrainedTokenizerBase, PaddingStrategy

from peft import (
    get_peft_config,
    PeftModel,
    PeftConfig,
    get_peft_model,
    LoraConfig,
    TaskType,
    prepare_model_for_kbit_training
)
import os

import random

In [ ]:
class args:
    train_data = './dataset/demo_train.csv'
    MAX_INPUT = 1024

In [ ]:
MODEL = 'google/gemma-2-9b-it'
config = AutoConfig.from_pretrained(MODEL, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(MODEL,
                                             config=config,
                                             #quantization_config=bnb_config,
                                             torch_dtype=torch.bfloat16,
                                             device_map="auto",
                                             trust_remote_code=True,
                                             attn_implementation='eager')

In [ ]:
peft_config = LoraConfig(
            task_type=TaskType.CAUSAL_LM,  # For sequence classification
            inference_mode=False,
            r=32,
            lora_alpha=64,
            lora_dropout=0.5,
            #bias = 'none',
            target_modules=['q_proj','k_proj','v_proj','o_proj'] #,
        )
model = get_peft_model(model, peft_config)

In [ ]:
for param in filter(lambda p: p.requires_grad, model.parameters()):
    param.data = param.data.to(torch.bfloat16)

In [ ]:
?LoraConfig

In [ ]:
for key in model.state_dict():
    print(f"{key}, {model.state_dict()[key].shape}, {model.state_dict()[key].dtype}")

In [ ]:
[i for i in model.parameters()]

In [ ]:
df_train = pd.read_csv(args.train_data).reset_index(drop = True)
#df_valid = pd.read_csv(args.valid_data).reset_index(drop = True)

In [ ]:
option_to_index = {option: idx for idx, option in enumerate('ABCDE')}

In [ ]:
def get_label(row):
    label = [idx for idx, option in enumerate(['winner_model_a','winner_model_b','winner_tie']) if row[option] == 1]
    return label[-1]

In [ ]:
df_train['label'] = df_train.apply(lambda x: get_label(x), axis = 1)

In [ ]:
def preprocess(example):
    first_sentence = [ "[CLS] " + example['prompt'] ] * 2
    second_sentences = [" #### " + example['prompt'] + " [SEP] " + example[option] + " [SEP]" for option in ['response_a','response_b']]
    tokenized_example = tokenizer(first_sentence, second_sentences, truncation='longest_first', 
                                  max_length=args.MAX_INPUT, add_special_tokens=False)
    tokenized_example['label'] = example['label']
    return tokenized_example

In [ ]:
def preprocess(example):
    sentences = [" #### " + example['prompt'] + " [SEP] " + example['response_a'] + " [SEP]" +  " #### " + example['prompt'] + " [SEP] " + example['response_b'] + " [SEP]"]
    tokenized_example = tokenizer(sentences, truncation=True, 
                                  max_length=args.MAX_INPUT, add_special_tokens=False)
    tokenized_example['label'] = example['label']
    return tokenized_example

In [ ]:
Review the user’s question and the corresponding response, along with two judgments. Determine which judgment is more accurate according to the rubric provided below. The rubric used for the initial judgments is as follows:
- Add 1 point if the response is relevant and provides some information related to
the user’s inquiry, even if it is incomplete or contains some irrelevant content.
- Add another point if the response addresses a substantial portion of the user’s question,
but does not completely resolve the query or provide a direct answer.
- Award a third point if the response answers the basic elements of the user’s question in a
useful way, regardless of whether it seems to have been written by an AI Assistant or if it
has elements typically found in blogs or search results.
- Grant a fourth point if the response is clearly written from an AI Assistant’s perspective,
addressing the user’s question directly and comprehensively, and is well-organized and helpful,
even if there is slight room for improvement in clarity, conciseness or focus.
- Bestow a fifth point for a response that is impeccably tailored to the user’s question
by an AI Assistant, without extraneous information, reflecting expert knowledge, and
demonstrating a high-quality, engaging, and insightful answer.


In [ ]:
dataset = datasets.Dataset.from_pandas(df_train)
MODEL = 'microsoft/deberta-v3-base'
tokenizer = AutoTokenizer.from_pretrained(MODEL)

In [ ]:
dataset

In [ ]:
tokenized_dataset = dataset.map(preprocess, remove_columns=['id', 'model_a', 'model_b', 'prompt', 'response_a', 'response_b'])# 

In [ ]:
tokenized_dataset

In [ ]:
tokenized_dataset['input_ids'][0]

In [ ]:
df_train.loc[40]

In [ ]:
df_train.loc[:1000,].reset_index(drop = True).to_csv('demo_train.csv')

In [ ]:
df_train.loc[1000:1200,].reset_index(drop = True).to_csv('demo_valid.csv')

In [ ]:
import os

from typing import Optional, Union
import pandas as pd, numpy as np, torch
from dataclasses import dataclass
from transformers import AutoTokenizer, AutoConfig
from transformers import EarlyStoppingCallback
from transformers.tokenization_utils_base import PreTrainedTokenizerBase, PaddingStrategy
from transformers import AutoModelForMultipleChoice, TrainingArguments, Trainer, RobertaForMultipleChoice, AutoModelForSequenceClassification, LlamaModel, LlamaForSequenceClassification, BitsAndBytesConfig
import argparse
from transformers import get_polynomial_decay_schedule_with_warmup, TrainerCallback
import datasets
from datasets import Dataset
from sklearn.metrics import log_loss
import torch.nn as nn
from peft import get_peft_config, PeftModel, PeftConfig, get_peft_model, LoraConfig, TaskType
    

In [ ]:
MODEL = 'meta-llama/llama-3-transformers-8b-hf-v1'
tokenizer = AutoTokenizer.from_pretrained(MODEL)
tokenizer.add_special_tokens({"pad_token":"<pad>"})

In [ ]:
tokenizer("<pad>")['input_ids'][0]

In [ ]:
128256 in tokenizer("<pad>")['input_ids']

In [ ]:
bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,  
        bnb_4bit_quant_type='nf4',
        bnb_4bit_compute_dtype=torch.bfloat16,
        bnb_4bit_use_double_quant=False
    )
    
#config = AutoConfig.from_pretrained(args.MODEL)
model = LlamaForSequenceClassification.from_pretrained(
    MODEL,
    num_labels=3,
    torch_dtype=torch.bfloat16,
    quantization_config=bnb_config,
    #config = config,
    device_map="auto")

In [ ]:
for name, param in model.named_parameters():
    print(f'Layer: {name}, dtype: {param.dtype}')

In [ ]:
bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,  
        bnb_4bit_quant_type='nf4',
        bnb_4bit_compute_dtype=torch.float16,
        bnb_4bit_use_double_quant=False
    )
    
#config = AutoConfig.from_pretrained(args.MODEL)
model = LlamaForSequenceClassification.from_pretrained(
    MODEL,
    num_labels=3,
    torch_dtype=torch.bfloat16,
    quantization_config=bnb_config,
    #config = config,
    device_map="auto")
model.config.pad_token_id = tokenizer.pad_token_id
model.resize_token_embeddings(len(tokenizer))

# config = AutoConfig.from_pretrained(MODEL)
# config.hidden_dropout_prob = args.dropout_rate
# config.attention_probs_dropout_prob = args.dropout_rate
# peft_config = LoraConfig(
#     task_type=TaskType.SEQ_CLS,  # For sequence classification
#     inference_mode=False,
#     r=16,
#     lora_alpha=16,
#     lora_dropout=0.1,
#     bias = 'none',
#     target_modules=["q_proj","k_proj","v_proj"]  # Target specific modules
# )
# model = get_peft_model(model, peft_config)

In [ ]:
for key in model.state_dict():
        print(f"{key}, {model.state_dict()[key].shape}, {model.state_dict()[key].dtype}")

In [ ]:
for name, param in model.named_parameters():
    print(f'Layer: {name}, dtype: {param.dtype}')

In [ ]:
for name, param in model.named_parameters():
    print(f'Layer: {name}, dtype: {param.dtype}')

In [ ]:
[i.dtype for i in model.parameters()]

In [ ]:
from utils import load_split_data

In [ ]:
df_train, df_valid = load_split_data('dataset/1k_mt_bench_human_judgments.json', 1, 3000, True, False)

In [ ]:
df_train, df_valid = load_split_data('dataset/lmsys-chatbot_arena_conversations-33k.csv', 2, 3000, True, False)

In [ ]:
df_valid

In [ ]:
df_train

In [ ]:
idx = 1
prompt_response = df_train.loc[idx,'prompt_response']
label = df_train.loc[idx,'label']

In [ ]:
print(prompt_response)
print("")
print(label)

In [ ]:
prompt_response

In [ ]:
tokenizer.decode([1,
 32006,
 887,
 526])

In [ ]:
tokenizer.decode([887])

In [ ]:
tokenizer.encode('<|system|>\nYou')

In [ ]:
tokenizer.encode('Apple\nBa')

In [ ]:
tokenizer.decode([396,
 18571,
 415,
 13,
 4548,
 7420,])

In [ ]:
tokenizer.decode([29933])

In [ ]:
print(templete_part1 + prompt_response + templete_part2 + templete_part3 + label)

In [ ]:
templete_part1 = "<|system|>\nYou are a helpful assistant good at judging conversations.<|end|>\n<|user|>\nHere are two question-answering dialogues. Compare two model performance on answering question, determine which is better.\n"
templete_part1_input_ids = tokenizer(text=templete_part1, add_special_tokens=True, padding=False)['input_ids']

templete_part2 = "\n###options\nA. Model A\nB. Model B\nC. Tie\n<|end|>\n"
templete_part2_input_ids = tokenizer(text=templete_part2, add_special_tokens=True, padding=False)['input_ids'][1:]

templete_part3 = "<|assistant|>\n"
templete_part3_input_ids = tokenizer(text=templete_part3, add_special_tokens=True, padding=False)['input_ids'][1:]

prompt_response_ids = tokenizer(text=prompt_response, add_special_tokens=True, truncation=True,
                                      max_length=3000, padding=False)['input_ids'][1:]


label_ids = tokenizer.encode(text=label, add_special_tokens=False)
input_ids = templete_part1_input_ids + prompt_response_ids + templete_part2_input_ids + templete_part3_input_ids + label_ids + [tokenizer.eos_token_id]
print(tokenizer.decode(input_ids))

In [ ]:
text = "Apple"
prompt_response = templete_part1 + text + templete_part2 + templete_part3 + label + tokenizer.eos_token
print(prompt_response)

In [ ]:
MODEL = 'microsoft/LLM-Research/Phi-3-mini-4k-instruct'
tokenizer = AutoTokenizer.from_pretrained(MODEL, trust_remote_code=True, truncation_side = 'left')

In [ ]:
tokenizer(prompt_response)

In [ ]:
tokenizer.eos_token_id

In [ ]:
tokenizer.eos_token

In [ ]:
tokenizer.bos_token

In [ ]:
tokenizer.decode([887])

In [ ]:
tokenizer('A',add_special_tokens=True, truncation=True, max_length=1024)['input_ids']

In [ ]:
tokenizer.pad_token_id

In [ ]:
tokenizer.pad_token

In [ ]:
tokenizer('<|user|>',add_special_tokens=True, truncation=True, max_length=1024)['input_ids']

In [ ]:
AutoModelForCausalLM.from_pretrained(MODEL)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2-7B-Instruct")

prompt = "Give me a short introduction to large language model."
messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": prompt}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)

In [ ]:
print(text)

In [ ]:
templete_part1 = "<|im_start|>system\nYou are a helpful assistant good at judging conversations.<|im_end|>\n<|im_start|>user\nHere are two question-answering dialogues. Compare two model performance on answering question, determine which is better.\n"
templete_part1_input_ids = tokenizer(text=templete_part1, add_special_tokens=True, padding=False)['input_ids']

templete_part2 = "\n###options\nA. Model A\nB. Model B\nC. Tie\n<|im_end|>\n"
templete_part2_input_ids = tokenizer(text=templete_part2, add_special_tokens=True, padding=False)['input_ids']
#print(f"templete_part2 is {templete_part2_input_ids}")
templete_part3 = "<|im_start|>assistant\n"
templete_part3_input_ids = tokenizer(text=templete_part3, add_special_tokens=True, padding=False)['input_ids']

prompt_response_ids = tokenizer(text=prompt_response, add_special_tokens=True, truncation=True,
                                      max_length=3000, padding=False)['input_ids']


label_ids = tokenizer.encode(text=label, add_special_tokens=False)
input_ids = templete_part1_input_ids + prompt_response_ids + templete_part2_input_ids + templete_part3_input_ids + label_ids + [tokenizer.eos_token_id]
print(tokenizer.decode(input_ids))

In [ ]:
tokenizer.decode(14374)

In [ ]:
tokenizer.pad_token,tokenizer.eos_token,

In [ ]:
tokenizer('A',add_special_tokens=True, truncation=True, max_length=1024)['input_ids']

In [ ]:
MODEL = 'Qwen/Qwen2-7B-Instruct'
config = AutoConfig.from_pretrained(MODEL, trust_remote_code=True)
tokenizer = AutoTokenizer.from_pretrained(MODEL, trust_remote_code=True, truncation_side = 'left')
bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,  
        bnb_4bit_quant_type='nf4',
        bnb_4bit_compute_dtype=torch.bfloat16,
        bnb_4bit_use_double_quant=True
    )
model = AutoModelForCausalLM.from_pretrained(MODEL,
                                             config=config,
                                             quantization_config=bnb_config,
                                             torch_dtype=torch.bfloat16,
                                             device_map="auto",
                                             trust_remote_code=True,
                                             attn_implementation='eager')

In [ ]:
model

In [ ]:
tmp = pd.read_json("dataset/kaggle-ultrafeedback-drop-duplicate-sample14k.json")
#tmp2 = pd.read_json("dataset/lmsys-chatbot_arena_conversations-33k.json")

In [ ]:
tmp = tmp.drop(columns = ['difference'])

In [ ]:
t = pd.concat([tmp,tmp2]).reset_index(drop = True)

In [ ]:
t.prompt[0]

In [ ]:
is_english(t.prompt[0][0])

In [ ]:
is_english(t.prompt[46969][0])

In [ ]:
from langdetect import detect

# 检测语言并过滤非英文行
def is_english(text):
    try:
        return detect(text) == 'en'
    except:
        return False

tmp = tmp[tmp['prompt'].apply(lambda x: is_english(x[0]))]

In [ ]:
tmp = tmp.reset_index(drop = True)

In [ ]:
tmp.to_json("dataset/kaggle-ultrafeedback-drop-duplicate-sample14k.json")

In [ ]:
ex = pd.read_csv("dataset/kaggle-ultrafeedback-drop-duplicate.csv")
tie = pd.read_csv("dataset/kaggle-ultrafeedback-ties-drop-duplicate.csv")
p = pd.read_csv("dataset/ultrafeedback_prediction.csv")

from utils import load_json
ex = load_json(ex)
tie = load_json(tie)

In [ ]:
total = pd.concat([tie,ex]).reset_index(drop = True)

In [ ]:
p

In [ ]:
p = p.rename(columns = {'winner_model_a':"p_winner_model_a", 'winner_model_b':"p_winner_model_b",  'winner_tie':"p_winner_tie"})

In [ ]:
final = pd.concat([total, p], axis = 1)

In [ ]:
final

In [ ]:
def get_p_label(row):
    a = row.p_winner_model_a
    b = row.p_winner_model_b
    c = row.p_winner_tie

    l = [a ,b, c]
    label = l.index(max(l))
    return label

In [ ]:
final['p_label'] = final.apply(get_p_label, axis = 1)

In [ ]:
def get_label(row):
    label = [idx for idx, option in enumerate(['winner_model_a','winner_model_b','winner_tie']) if row[option] == 1]
    return label[-1]

final['label'] = final.apply(get_label, axis = 1)

In [ ]:
filter = final.loc[final.p_label == final.label,:].reset_index(drop = True)

In [ ]:
threshold = 0.9
filter_list = (filter.p_winner_model_a >= threshold) | (filter.p_winner_model_b >= threshold) | (filter.p_winner_tie >= threshold)
filter = filter.loc[filter_list,:].reset_index(drop = True)

In [ ]:
filter.prompt.values[0][0]

In [ ]:
from langdetect import detect

# 检测语言并过滤非英文行
def is_english(text):
    try:
        return detect(text) == 'en'
    except:
        return False

filter_only_english = filter[filter['prompt'].apply(lambda x: is_english(x[0][:30]))].reset_index(drop = True)

In [ ]:
save_columns = ['prompt', 'model_a', 'model_b', 'winner_model_a', 'winner_model_b', 'winner_tie', 'response_a', 'response_b', 'id']
filter[save_columns].to_json(f"dataset/70k_filter_threshold{threshold}.json", index = False)
filter_only_english[save_columns].to_json(f"dataset/70k_filter_only_english_threshold{threshold}.json", index = False)

In [ ]:
filter[save_columns]

In [ ]:
filter.label.value_counts()

In [ ]:
df_train, df_valid = load_split_data('dataset/train.csv', 2, 3000, True, False)

In [ ]:
df_train

In [ ]:
print(df_train.prompt_response.values[0])

In [ ]:
print(df_train.prompt_response.values[1])

In [ ]:
print(df_train.label.values[1])

In [ ]:
check = pd.read_json("dataset/70k_filter_only_english_threshold0.9.json")

In [ ]:
check.prompt.values[0]

In [ ]:
data_path = "dataset/kaggle-ultrafeedback-drop-duplicate-sample20k_least_similar_by_prompt_same_prediction_thr90.csv"
t = pd.read_csv(data_path)
t['id'] = [randint(10000,99999) + i for i in range(len(t))]
t.to_csv(data_path, index=False)

In [ ]:
t = pd.read_json("dataset/kaggle-ultrafeedback-drop-duplicate-sample20k_most_similar_by_prompt_same_prediction_thr90.json")

In [ ]:
#
train , valid = load_split_data('dataset/train.csv', 2, 2300, True, True, False)

In [ ]:
train , valid = load_split_data('dataset/train_sample10k_switch.json', 2, 2300, True, False, False)

In [ ]:
train_id = train.id.to_list()
valid_id = valid.id.to_list()

In [ ]:
data = pd.read_csv('dataset/train.csv')
data = load_json(data)

In [ ]:
data['prompt_str'] = data['prompt'].astype(str)

In [ ]:
from sklearn.model_selection import train_test_split
unique_prompts = data['prompt_str'].unique()
train_prompts, valid_prompts = train_test_split(unique_prompts, test_size=0.1, random_state=42)

In [ ]:
train_prompts_set = set(train_prompts)
valid_prompts_set = set(valid_prompts)

In [ ]:
# 根据划分的 prompt 获取对应的行
train = data[data['prompt_str'].isin(train_prompts_set)]
valid = data[data['prompt_str'].isin(valid_prompts_set)]

In [ ]:
train = data.loc[data.id.isin(train_id)].reset_index(drop = True)
valid = data.loc[data.id.isin(valid_id)].reset_index(drop = True)

In [ ]:
[i for i in train_id if i in valid_id]

In [ ]:
len(train_id)

In [ ]:
t = pd.read_json('dataset/train_sample10k_switch.json')

In [ ]:
len([i for i in t.prompt.values.tolist() if i in valid.prompt.values.tolist()])

In [ ]:
search = [['I read 60 pages of a book on Monday and 1/4 of the book on Tuesday. I completed the remaining 1/8 of the book on Wednesday. How many total pages are in the book?']]
train.loc[train.prompt.isin(search)]

In [ ]:
valid.loc[valid.prompt.isin(search)]

In [ ]:
[i for i in t.prompt.values.tolist() if i in valid.prompt.values.tolist()]

In [ ]:
len([i for i in train.prompt.values.tolist() if i in valid.prompt.values.tolist()])

In [ ]:
t.prompt.values.tolist()[0]

In [ ]:
train , valid = load_split_data('dataset/train.csv', 2, 2300, True, True, False)
valid_id = valid.id.tolist()
train_id = train.id.tolist()

In [ ]:
data = pd.read_csv('dataset/train.csv')
data = load_json(data)

In [ ]:
s = data.loc[data.id.isin(train_id)]

In [ ]:
s = s.sample(10000).reset_index(drop = True)

In [ ]:
s_id = s.id.to_list()
len([i for i in s_id if i in valid_id])

In [ ]:
tmp_valid = data.loc[data.id.isin(valid_id)].reset_index(drop = True)
[i for i in s.prompt.values.tolist() if i in tmp_valid.prompt.values.tolist()]

In [ ]:
ex_33 = pd.read_csv('dataset/lmsys-chatbot_arena_conversations-33k.csv')
ex_33 = load_json(ex_33)

In [ ]:
idx = [idx for idx, i in enumerate(tmp_valid.prompt.values.tolist()) if i in ex_33.prompt.values.tolist()]

In [ ]:
same_prompt_in_valid = tmp_valid.iloc[idx,:].reset_index(drop = True)
not_same_prompt_in_valid = tmp_valid.iloc[~tmp_valid.index.isin(idx),:].reset_index(drop = True)

In [ ]:
assert len(not_same_prompt_in_valid) + len(same_prompt_in_valid) == len(tmp_valid)

In [ ]:
same_prompt_in_valid.to_json("dataset/same_prompt_in_valid.json", index = False)
not_same_prompt_in_valid.to_json("dataset/not_same_prompt_in_valid.json", index = False)

In [ ]:
same_prompt_in_valid.sort_values(by = ['prompt'])

In [ ]:
idx_ex = [idx for idx, i in enumerate(ex_33.prompt.values.tolist()) if i in tmp_valid.prompt.values.tolist()]

In [ ]:
ex_33.iloc[idx_ex,:].reset_index(drop = True).sort_values(by = ['prompt'])

In [ ]:
data = pd.read_json("dataset/pass/demo_A2B2C.json")

In [ ]:
s = data.sample(int(len(data) * 0.3)).reset_index(drop = True)

In [ ]:
s['id'] = [randint(100000,999999) + i for i in range(len(s))]

In [ ]:
def switch(row):
    response_a = row.response_a
    response_b = row.response_b

    row.response_a = response_b
    row.response_b = response_a
    
    return row

In [ ]:
s = s.apply(switch, axis = 1)
label_switch = {0:1, 1:0}
s.loc[s.winner_tie !=1, 'winner_model_a'] = s.loc[s.winner_tie !=1, 'winner_model_a'].map(label_switch)
s.loc[s.winner_tie !=1, 'winner_model_b'] = s.loc[s.winner_tie !=1, 'winner_model_b'].map(label_switch)

In [ ]:
final = pd.concat([s, data]).reset_index(drop = True)
final['id'] = [randint(1000,999999) + i for i in range(len(final))]

In [ ]:
#save_columns = ['prompt', 'model_a', 'model_b', 'winner_model_a', 'winner_model_b', 'winner_tie', 'response_a', 'response_b', 'id']
final.to_json(f"dataset/pass/demo_A2B2C_tta.json", index = False)

In [ ]:
s.to_json("dataset/train_sample10k_switch.json", index = False)

In [ ]:
check = pd.read_json("dataset/pass/demo_A2B2C_tta.json")

In [ ]:
check

In [ ]:
print(check.prompt.values[1][0])

In [ ]:
t.loc[t.response_a == t.response_b, 'winner_tie'].value_counts()

In [ ]:
t.loc[t.response_a=='["Hyderabad"]', ]

In [ ]:
t.loc[(t.response_a == '[null]')]

In [ ]:
t.loc[(t.response_a == t.response_b) & (t.winner_tie != 1),]

In [ ]:
t.loc[(t.response_a == t.response_b)]

In [ ]:
t.loc[3844:3847,:]

In [ ]:
data = pd.read_csv('dataset/train.csv')
data = load_json(data)

ex_33 = pd.read_csv('dataset/lmsys-chatbot_arena_conversations-33k.csv')
ex_33 = load_json(ex_33)

In [ ]:
'''
1、找出train里面不与33k重复部分
2、不重复的部分再划分
'''



In [ ]:
set_prompt_response = []
for i in data.itertuples():
    prompt_response = i.prompt + i.response_a + i.response_b
    set_prompt_response.append(set(prompt_response))
data['set_prompt_response'] = set_prompt_response    

In [ ]:
set_prompt_response = []
for i in ex_33.itertuples():
    prompt_response = i.prompt + i.response_a + i.response_b
    set_prompt_response.append(set(prompt_response))
ex_33['set_prompt_response'] = set_prompt_response  

In [ ]:
idx = [idx for idx, i in enumerate(data.set_prompt_response.values) if i in ex_33.set_prompt_response.values]

In [ ]:
same = data.loc[idx,:].reset_index(drop = True)

In [ ]:
ex_33.loc[ex_33.set_prompt_response == same.set_prompt_response.values[0],:]

In [ ]:
not_same = data.loc[~data.index.isin(idx),:].reset_index(drop = True)

In [ ]:
assert len(not_same) + len(same) == len(data)

In [ ]:
len([idx for idx, i in enumerate(not_same.set_prompt_response.values) if i in ex_33.set_prompt_response.values])

In [ ]:
len([idx for idx, i in enumerate(not_same.set_prompt_response.values) if i in same.set_prompt_response.values])

In [ ]:
unique_sets = not_same['set_prompt_response'].drop_duplicates().reset_index(drop=True)
# 将唯一集合进行随机划分
unique_sets = unique_sets.sample(frac=1, random_state=42).reset_index(drop=True)
midpoint = len(unique_sets) // 10
set1 = unique_sets.iloc[:midpoint]
set2 = unique_sets.iloc[midpoint:]

In [ ]:
# 根据划分结果从原数据集中提取对应的行
valid = not_same[not_same['set_prompt_response'].isin(set1)].reset_index(drop=True)
train_subset = not_same[not_same['set_prompt_response'].isin(set2)].reset_index(drop=True)
assert len(valid) + len(train_subset) == len(not_same)
assert len(valid) + len(train_subset) + len(same) == len(data)

In [ ]:
len([idx for idx, i in enumerate(valid.set_prompt_response.values) if i in train_subset.set_prompt_response.values])

In [ ]:
len([idx for idx, i in enumerate(valid.set_prompt_response.values) if i in same.set_prompt_response.values])

In [ ]:
train_exclude_valid = pd.concat([train_subset, same]).reset_index(drop=True) #train 里面排除valid
len([idx for idx, i in enumerate(valid.set_prompt_response.values) if i in train_exclude_valid.set_prompt_response.values])

In [ ]:
assert len(valid) + len(train_exclude_valid) == len(data)

In [ ]:
train_33k = pd.concat([train_subset, ex_33]).reset_index(drop=True)

In [ ]:
train_subset = train.drop(columns = ['set_prompt_response'])
valid = valid.drop(columns = ['set_prompt_response'])
train_exclude_valid = train_exclude_valid.drop(columns = ['set_prompt_response'])
train_33k = train_33k.drop(columns = ['set_prompt_response'])

In [ ]:
train_subset.to_json("dataset/non_overlap/train_subset.json", index = False)
valid.to_json("dataset/non_overlap/valid.json", index = False)
train_exclude_valid.to_json("dataset/non_overlap/train_exclude_valid.json", index = False)
train_33k.to_json("dataset/non_overlap/train_33k.json", index = False)

In [ ]:
# 检查
train_subset = pd.read_json("dataset/non_overlap/train_subset.json")
valid = pd.read_json("dataset/non_overlap/valid.json")
train_exclude_valid = pd.read_json("dataset/non_overlap/train_exclude_valid.json")
train_33k = pd.read_json("dataset/non_overlap/train_33k.json")

In [ ]:
def get_set_prompt_response(data):
    set_prompt_response = []
    for i in data.itertuples():
        prompt_response = i.prompt + i.response_a + i.response_b
        set_prompt_response.append(set(prompt_response))
    data['set_prompt_response'] = set_prompt_response  
    return data

In [ ]:
train_subset = get_set_prompt_response(train_subset)
valid = get_set_prompt_response(valid)
train_exclude_valid = get_set_prompt_response(train_exclude_valid)
train_33k = get_set_prompt_response(train_33k)

In [ ]:
#valid和任何都不重合
assert len([idx for idx, i in enumerate(valid.set_prompt_response.values) if i in train_subset.set_prompt_response.values]) == 0
assert len([idx for idx, i in enumerate(valid.set_prompt_response.values) if i in train_exclude_valid.set_prompt_response.values]) == 0
assert len([idx for idx, i in enumerate(valid.set_prompt_response.values) if i in train_33k.set_prompt_response.values]) == 0

In [ ]:
train_33k.prompt.values[0]

In [ ]:
# 提取唯一的 prompt 进行划分
not_same['prompt_str'] = not_same['prompt'].astype(str)
unique_prompts = data['prompt_str'].unique()
train_prompts, valid_prompts = train_test_split(unique_prompts, test_size=0.1, random_state=42)

train_prompts_set = set(train_prompts)
valid_prompts_set = set(valid_prompts)

# 根据划分的 prompt 获取对应的行
train = data[data['prompt_str'].isin(train_prompts_set)].reset_index(drop = True)
valid = data[data['prompt_str'].isin(valid_prompts_set)].reset_index(drop = True)
train = train.drop(columns = ['prompt_str'])
valid = valid.drop(columns = ['prompt_str'])

In [ ]:
train_33k = pd.read_json("dataset/non_overlap/train_33k.json")

In [ ]:
s = train_33k.sample(15000).reset_index(drop = True)

In [ ]:
s = s.apply(switch, axis = 1)
label_switch = {0:1, 1:0}
s.loc[s.winner_tie !=1, 'winner_model_a'] = s.loc[s.winner_tie !=1, 'winner_model_a'].map(label_switch)
s.loc[s.winner_tie !=1, 'winner_model_b'] = s.loc[s.winner_tie !=1, 'winner_model_b'].map(label_switch)

In [ ]:
s['id'] = [randint(100000,999999) + i for i in range(len(s))]

In [ ]:
s.to_json("dataset/non_overlap/train_33k_switch_15k.json", index=False)

In [ ]:
def get_set_prompt_response(data):
    set_prompt_response = []
    for i in data.itertuples():
        prompt_response = i.prompt + i.response_a + i.response_b
        set_prompt_response.append(set(prompt_response))
    data['set_prompt_response'] = set_prompt_response  
    return data

In [ ]:
valid = get_set_prompt_response(valid)
s = get_set_prompt_response(s)

In [ ]:
train_33k.loc[train_33k.response_b.isin([['Three times 78234 is 234,692.']])]

In [ ]:
len([idx for idx, i in enumerate(valid.set_prompt_response.values) if i in s.set_prompt_response.values])

In [ ]:
pd.read_json("dataset/non_overlap/train_33k_switch_15k.json").response_a.values[1]

# prompt3

In [ ]:
def prompt_2(data, max_length, if_train):
    '''
    超过max length新开一行，label不变
    #Prompt1
    xxxx
    #Response
    ##Model A
    xxxx
    ##Model B
    xxxx
    
    #Prompt2
    #Response
    ##Model A
    xxxx
    ##Model B
    xxxx
    '''

    data['prompt_response'] = "#Prompt\n" + data['prompt'] + "\n\n" + "#Response\n" + "##Model A\n" + data['response_a'] + "\n\n" + "##Model B\n" + data['response_b']

    prompt_response = []
    ids = []
    labels = []
    text_length = 0
    for idx, row in tqdm(data.iterrows(), total=len(data)):
        text = row['prompt_response']
        if if_train:
            label = row['label']
        id = row['id']
        if id not in ids:
            #第一次出现
            prompt_response.append(text)
            text_length = len(text.split(" "))
            ids.append(id)
            if if_train:
                labels.append(label)
        else:
            text_length += len(text.split(" "))
            if text_length <= max_length:
                #取上一个text出来，合并后替换
                text = prompt_response[-1] + "\n\n" + text
                prompt_response[-1] = text
            else:
                #另一起一行
                prompt_response.append(text)
                text_length = len(text.split(" "))
                ids.append(id)
                if if_train:
                    labels.append(label)
    if if_train:           
        data = pd.DataFrame({'id': ids, 'prompt_response': prompt_response, "label": labels})
    else:
        data = pd.DataFrame({'id': ids, 'prompt_response': prompt_response})
    return data

def get_label(row):
    label = [idx for idx, option in enumerate(['winner_model_a','winner_model_b','winner_tie']) if row[option] == 1]
    if label[-1] == 0:
        return 'A'
    elif label[-1] == 1:
        return 'B'
    else:
        return 'C'
    return label[-1]

In [ ]:
data = pd.read_json("dataset/non_overlap/train_subset.json")
if_train = True

In [ ]:
#seperate prompt-response
data = data.explode(['prompt','response_a','response_b']).reset_index(drop = True)

#prepare label
if if_train:
    data['label'] = data.apply(lambda x: get_label(x), axis = 1)

data = data.fillna('None')
data['response_a'] = data['response_a'].apply(lambda x: 'None' if len(x)==0 else x)
data['response_b'] = data['response_b'].apply(lambda x: 'None' if len(x)==0 else x)

In [ ]:
data['prompt_response'] = "#Prompt\n" + data['prompt'] + "\n\n" + "#Response\n" + "##Model A\n" + data['response_a'] + "\n\n" + "##Model B\n" + data['response_b']

In [ ]:
data

In [ ]:
'''
反转dataframe
用栈，先进后出，超过max length就清空
'''
def prompt_3(data, max_length, if_train):
    '''
    超过max length新开一行，label不变
    从后往前拼接
    #Prompt1
    xxxx
    #Response
    ##Model A
    xxxx
    ##Model B
    xxxx
    
    #Prompt2
    #Response
    ##Model A
    xxxx
    ##Model B
    xxxx
    '''

    data['prompt_response'] = "#Prompt\n" + data['prompt'] + "\n\n" + "#Response\n" + "##Model A\n" + data['response_a'] + "\n\n" + "##Model B\n" + data['response_b']
    data = data.iloc[::-1].reset_index(drop = True)#反转
    prompt_response = []
    ids = []
    labels = []
    text_length = 0
    for idx, row in tqdm(data.iterrows(), total=len(data)):
        text = row['prompt_response']
        if if_train:
            label = row['label']
        id = row['id']
        if id not in ids:
            #第一次出现
            prompt_response.append(text)
            text_length = len(text.split(" "))
            ids.append(id)
            if if_train:
                labels.append(label)
        else:
            text_length += len(text.split(" "))
            if text_length <= max_length:
                #取上一个text出来，合并后替换
                text = text + "\n\n" + prompt_response[-1]
                prompt_response[-1] = text
            else:
                #另一起一行
                prompt_response.append(text)
                text_length = len(text.split(" "))
                ids.append(id)
                if if_train:
                    labels.append(label)
    if if_train:           
        data = pd.DataFrame({'id': ids, 'prompt_response': prompt_response, "label": labels})
        data = data.iloc[::-1].reset_index(drop = True)#反转
    else:
        data = pd.DataFrame({'id': ids, 'prompt_response': prompt_response})
        data = data.iloc[::-1].reset_index(drop = True)#反转
    return data

In [ ]:
prompt3 = prompt_3(data, 1900, True)
prompt2 = prompt_2(data, 1900, True)

In [ ]:
prompt3

In [ ]:
check_p3 = prompt3.loc[prompt3.id == 2846599172]
#cehck_data = data.loc[data.id == 2846599172]

check_p2 = prompt2.loc[prompt3.id == 2846599172]
#cehck_data = data.loc[data.id == 2846599172]

In [ ]:
cehck_data

In [ ]:
print(check_p3.prompt_response.values[0])

In [ ]:
print(check_p3.prompt_response.values[1])

In [ ]:
len(prompt3.prompt_response.values[1])

In [ ]:
len(prompt3.prompt_response.values[0])

In [ ]:
print(data.prompt_response.values[0])

In [ ]:
print(data.prompt_response.values[1])

In [ ]:
check_p3.prompt_response.apply(lambda x:len(x.split(" ")))

In [ ]:
check_p2.prompt_response.apply(lambda x:len(x.split(" ")))

In [ ]:
data = pd.read_json("dataset/mt_bentch_3k.json")

In [ ]:
data = data.loc[data.type == 'human'].reset_index(drop = True)

In [ ]:
data.to_json("dataset/mt_bentch_human.json", index=False)

In [ ]:

def get_label(row):
    label = [idx for idx, option in enumerate(['winner_model_a','winner_model_b','winner_tie']) if row[option] == 1]
    print(label)
    if label[-1] == 0:
        return 'A'
    elif label[-1] == 1:
        return 'B'
    else:
        return 'C'
    return label[-1]
data['label'] = data.apply(lambda x: get_label(x), axis = 1)

In [ ]:
data['total'] = data.winner_model_a + data.winner_model_b + data.winner_tie

In [ ]:
data.loc[data.total == 0]

In [ ]:
df_valid, _ = load_split_data('dataset/non_overlap/valid.json', 3, 1900, True, False, False, True, False)

In [ ]:
df_train, _ = load_split_data('dataset/1M/15k_preds.csv', 3, 1900, True, False, False, True, False)

In [ ]:
df_train.label.value_counts().tolist()

In [ ]:
from tqdm import tqdm
def get_text_length(text):
    '''
    不用空格分隔的文本, text length = len
    不用空格分隔的一般tokenizer后长度类似，所以还可以缩小
    空格分隔的，len(text.split(" "))
    '''
    length1 = len(text)
    length2 = len(text.split(" "))
    #远超过
    if length1 >= length2 * 30 and length1>= 300:
        return length1 * 0.75
    return length2
    
def prompt_3(data, max_length, if_train):
    '''
    超过max length新开一行，label不变
    从后往前拼接
    #Prompt1
    xxxx
    #Response
    ##Model A
    xxxx
    ##Model B
    xxxx
    
    #Prompt2
    #Response
    ##Model A
    xxxx
    ##Model B
    xxxx
    '''

    data['prompt_response'] = "#Prompt\n" + data['prompt'] + "\n\n" + "#Response\n" + "##Model A\n" + data['response_a'] + "\n\n" + "##Model B\n" + data['response_b']
    data = data.iloc[::-1].reset_index(drop = True)#反转
    prompt_response = []
    ids = []
    labels = []
    #只有一种可能会超出max length：
    #单条的prompt和reponse加在一起超出max length
    over_max_length = [] #是否有超出max length的部分
    overflow_prompt = []
    overflow_response_a = [] #超出max length的部分
    overflow_response_b = [] #超出max length的部分
    text_length = 0
    for idx, row in tqdm(data.iterrows(), total=len(data)):
        text = row['prompt_response']
        response_a = row['response_a']
        response_b = row['response_b']
        prompt = row['prompt']
        id = row['id']
        
        if if_train:
            label = row['label']
        
        if id not in ids:
            #第一次出现
            prompt_response.append(text)
            text_length = get_text_length(text)
            ids.append(id)
            if if_train:
                labels.append(label)
            if text_length > max_length:
                over_max_length.append(1)
                overflow_prompt.append(prompt)
                overflow_response_a.append(response_a)
                overflow_response_b.append(response_b)
            else:
                over_max_length.append(0)
                overflow_prompt.append(None)
                overflow_response_a.append(None)
                overflow_response_b.append(None)
        
        else:
            text_length += get_text_length(text)
            if text_length <= max_length:
                #取上一个text出来，合并后替换
                text = text + "\n\n" + prompt_response[-1]
                prompt_response[-1] = text
                over_max_length[-1] = 0
                overflow_prompt[-1] = None
                overflow_response_a[-1] = None
                overflow_response_b[-1] = None
                
            else:
                #另一起一行
                prompt_response.append(text)
                text_length = get_text_length(text)
                ids.append(id)
                
                if if_train:
                    labels.append(label)
                    
                #另起一行但超出场合都
                if text_length > max_length:
                    over_max_length.append(1)
                    overflow_prompt.append(prompt)
                    overflow_response_a.append(response_a)
                    overflow_response_b.append(response_b)
                else:
                    over_max_length.append(0)
                    overflow_prompt.append(None)
                    overflow_response_a.append(None)
                    overflow_response_b.append(None)
                    
                
                    
    if if_train:           
        data = pd.DataFrame({'id': ids, 'prompt_response': prompt_response, "label": labels, 'overflow_prompt':overflow_prompt, 'over_max_length': over_max_length, 'overflow_response_a': overflow_response_a, 'overflow_response_b': overflow_response_b})
        data = data.iloc[::-1].reset_index(drop = True)#反转
    else:
        data = pd.DataFrame({'id': ids, 'prompt_response': prompt_response, 'over_max_length': over_max_length, 'overflow_prompt':overflow_prompt, 'overflow_response_a': overflow_response_a, 'overflow_response_b': overflow_response_b})
        data = data.iloc[::-1].reset_index(drop = True)#反转
    return data

In [ ]:
data = pd.read_json("dataset/non_overlap/train_33k.json")

In [ ]:
import re

def is_english_text(text):
    # 使用正则表达式找到所有英文字母
    english_letters = re.findall(r'[a-zA-Z]', text)
    
    # 如果英文字母的比例超过一定阈值（例如 70%），则认为主要内容是英文
    english_ratio = len(english_letters) / len(text)
    
    return english_ratio > 0.2

# 测试函数
text = "This is a test text with some English words and 一些中文字符."
print(is_english_text(text))  # 输出: True


In [ ]:
data = data.explode(['prompt','response_a','response_b']).reset_index(drop = True)

In [ ]:
data = data.fillna('None')
data['response_a'] = data['response_a'].apply(lambda x: 'None' if len(x)==0 else x)
data['response_b'] = data['response_b'].apply(lambda x: 'None' if len(x)==0 else x)

# data['response_a'] = data['response_a'].apply(lambda x: ["None" if i is None else i for i in x])
# data['response_b'] = data['response_b'].apply(lambda x: ["None" if i is None else i for i in x])

In [ ]:
data

In [ ]:
data['is_en'] = data['prompt'].apply(lambda x: is_english_text(x[0]))

In [ ]:
non_en =  data.loc[~data.is_en].reset_index(drop = True)

In [ ]:
data['str_len'] = data['prompt'].apply(len) + data['response_a'].apply(len) + data['response_b'].apply(len)
data['split_len'] = data['prompt'].apply(lambda x: len(x.split(" "))) + data['response_a'].apply(lambda x: len(x.split(" ")))  + data['response_b'].apply(lambda x: len(x.split(" "))) 

In [ ]:
# data['str_len'] = data['prompt'].apply(lambda x: sum([len(i) for i in x])) + data['response_a'].apply(lambda x: sum([len(i) for i in x])) + data['response_b'].apply(lambda x: sum([len(i) for i in x]))
# data['split_len'] = data['prompt'].apply(lambda x: sum([len(i.split(" ")) for i in x])) + data['response_a'].apply(lambda x: sum([len(i.split(" ")) for i in x])) + data['response_b'].apply(lambda x: sum([len(i.split(" ")) for i in x]))

In [ ]:
idx = (data.str_len >= data.split_len * 30 ) & ( data.str_len >= 500 )
non_en = data.loc[idx].reset_index(drop = True)


In [ ]:
non_en

In [ ]:
tmp = data.loc[data.id == 2789396693]
tmp

In [ ]:
t = prompt_3(tmp, 1900, False)
t

In [ ]:
print(t.prompt_response.values[0])

In [ ]:
MODEL = 'google/gemma-2-9b-it'
config = AutoConfig.from_pretrained(MODEL, trust_remote_code=True)
tokenizer = AutoTokenizer.from_pretrained(MODEL, trust_remote_code=True, truncation_side = 'left')

In [ ]:
len(tokenizer(t.prompt_response.values[0])['input_ids'])

In [ ]:
print(t.prompt_response.values[0])

In [ ]:
tokenizer.decode(235441)

In [ ]:
tokenizer(t.prompt_response.values[0])['input_ids']

In [ ]:
from utils_v2 import load_split_data
data_path = "dataset/non_overlap/train_33k.json"
prompt_type = 3
MAX_INPUT = 1900
if_train = True
split = False
if_drop_duplicate = True
keep = 'last'
base_model = 'google/gemma-2-9b-it'
model_path = "output/misunderstood-flower-508/checkpoint-5459_8857"
MAX_LENGTH = MAX_INPUT

df_train , df_valid = load_split_data(data_path, prompt_type, MAX_INPUT, if_train, split, False, if_drop_duplicate, 'last', base_model)
test = df_train

In [ ]:
text = test.loc[test.id == 2789396693].prompt_response.values
print(tokenizer.decode(text[0])),len(text[0])


In [ ]:
test['label'] = test['label'].apply(tokenizer.decode)

In [ ]:
test.loc[test.label == 'C'].length.median()

In [ ]:
test.loc[test.label == 'A'].length.median()

In [ ]:
test.loc[test.label == 'B'].length.median()

In [ ]:
m = test.loc[test.label == 'C'].length.median()
idx = ((test.label == 'C') & ((test.length > m - 100) & (test.length < m + 300))) | ((test.label == 'C') & (test.length == 1916))
filter_tie = test.loc[idx].reset_index(drop = True)

In [ ]:
filter_tie

In [ ]:
test.loc[test.length == 1916]

In [ ]:
data = pd.read_json("dataset/non_overlap/train_33k_switch_10k.json")

In [ ]:
data.loc[data.id == '5cf1be2e58964aada5f803d3845da7af']

In [ ]:
test['length'] = test['prompt_response'].apply(len)

In [ ]:
def tokenize(row, tokenizer):

    now_data = row
    response_a = row['response_a']
    response_a_input_ids = tokenizer(text=response_a, add_special_tokens=False, padding=False)['input_ids']
    row['response_a_input_ids'] = response_a_input_ids
    
    response_b = row['response_b']
    response_b_input_ids = tokenizer(text=response_b, add_special_tokens=False, padding=False)['input_ids']
    row['response_b_input_ids'] = response_b_input_ids
    
    prompt = row['prompt']
    prompt_input_ids = tokenizer(text=prompt, add_special_tokens=False, padding=False)['input_ids']
    row['prompt_input_ids'] = prompt_input_ids
    
    label = now_data['label']
    label_ids = tokenizer.encode(text=label, add_special_tokens=False)
    row['label_ids'] = label_ids

    return row

def get_label(row):
    label = [idx for idx, option in enumerate(['winner_model_a','winner_model_b','winner_tie']) if row[option] == 1]
    if label[-1] == 0:
        return 'A'
    elif label[-1] == 1:
        return 'B'
    else:
        return 'C'
    return label[-1]

In [ ]:
tqdm.pandas(desc = 'pandas bar')
#seperate prompt-response
data = data.explode(['prompt','response_a','response_b']).reset_index(drop = True)

#prepare label
if if_train:
    data['label'] = data.apply(lambda x: get_label(x), axis = 1)

data = data.fillna('None')
data['response_a'] = data['response_a'].apply(lambda x: 'None' if len(x)==0 else x)
data['response_b'] = data['response_b'].apply(lambda x: 'None' if len(x)==0 else x)

#分词
data = data.progress_apply(lambda x: tokenize(x, tokenizer), axis = 1)

In [ ]:
data['a_length'] = data['response_a_input_ids'].apply(len)
data['b_length'] = data['response_b_input_ids'].apply(len)
data['p_length'] = data['prompt_input_ids'].apply(len)

In [ ]:
data

In [ ]:
'''
A赢，但A明显短于B
B赢，但B明显短于A
打平，但两者长度差距很大
'''

idx1 = (data.label == 'A') & (data.a_length * 2 < data.b_length) 
idx2 = (data.label == 'B') & (data.a_length > data.b_length * 2) 
idx3 = (data.label == 'C') & ((data.a_length > data.b_length * 3) | (data.a_length * 3 < data.b_length))
idx4 = (data.a_length > 1000) | (data.b_length > 1000) | (data.p_length > 1000)
final = data[~(idx3 | idx1 | idx2)].reset_index(drop = True)
final
                                             

In [ ]:
ids = final.id.unique()

In [ ]:
ids.shape

In [ ]:
o = pd.read_json("dataset/non_overlap/train_33k_switch_10k.json")

In [ ]:
final = o.loc[o.id.isin(ids)].reset_index(drop = True)

In [ ]:
final

In [ ]:
o.loc[o.id.isin(ids)].reset_index(drop = True)

In [ ]:
def switch(row):
    response_a = row.response_a
    response_b = row.response_b

    row.response_a = response_b
    row.response_b = response_a
    
    return row

In [ ]:
final = final.apply(switch, axis = 1)
label_switch = {0:1, 1:0}
final.loc[final.winner_tie !=1, 'winner_model_a'] = final.loc[final.winner_tie !=1, 'winner_model_a'].map(label_switch)
final.loc[final.winner_tie !=1, 'winner_model_b'] = final.loc[final.winner_tie !=1, 'winner_model_b'].map(label_switch)

In [ ]:
final.to_json("dataset/non_overlap/train_33k_switch_10k_filter_diff.json",index=False)

In [ ]:
pd.read_json("dataset/non_overlap/train_33k_switch_10k_filter_diff.json")

In [ ]:
final = pd.read_json("dataset/non_overlap/train_33k_switch_dif_length_13k.json")
data = pd.read_json("dataset/non_overlap/train_33k.json")

In [ ]:
'''
单个id有超过两条以上的对话，就将都为None的丢掉
否则保留
'''
idx = data.response_b.isna() & data.response_a.isna()#都为none
t = data[idx].id.unique()
all_none = data.loc[data.id.isin(t)].reset_index(drop = True)
count = all_none.id.value_counts().reset_index()
drop_id = count.loc[count['count']>1].id.unique()
filter_idx = data.id.isin(drop_id) & idx
data = data[~filter_idx].reset_index(drop = True)

In [ ]:
data

In [ ]:
data.loc[data.id == 3939156539]

In [ ]:
data = pd.read_json("dataset/pretrain/korean_arena.json")

In [ ]:
idx = (data.winner_model_a == 0) &(data.winner_model_b == 0) & (data.winner_tie == 0)
data[idx]

In [ ]:
from utils_v2 import load_split_data
data_path = "dataset/pretrain/math_2k7_arena.json"
prompt_type = 3
MAX_INPUT = 1900
if_train = True
split = False
if_drop_duplicate = True
keep = 'last'
base_model = 'google/gemma-2-9b-it'
model_path = "output/misunderstood-flower-508/checkpoint-5459_8857"
MAX_LENGTH = MAX_INPUT

df_train , df_valid = load_split_data(data_path, prompt_type, MAX_INPUT, if_train, split, False, if_drop_duplicate, 'last', base_model)
test = df_train